## Домашнее задание 12

1. Взять датасет https://huggingface.co/datasets/merionum/ru_paraphraser, решить задачу парафраза
2. (дополнительно необязательная задача)на выбор взять
- https://huggingface.co/datasets/sberquad
- https://huggingface.co/datasets/blinoff/medical_qa_ru_data
натренировать любую модель для вопросно-ответной системы
как альтернатива можно взять любой NER датасет из https://github.com/natasha/corus#reference и обучить NER

## Решение

1. Установим и импортируем необходимые библиотеки:

In [1]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 27.1 MB/s 
     |████████████████████████████████| 431 kB 68.1 MB/s 
     |████████████████████████████████| 120 kB 72.7 MB/s 
     |████████████████████████████████| 6.6 MB 44.1 MB/s 
     |████████████████████████████████| 115 kB 60.4 MB/s 
     |████████████████████████████████| 212 kB 65.9 MB/s 
     |████████████████████████████████| 127 kB 57.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import unicodedata
import re
import random
import transformers
import os
import io
import time
import torch
import tqdm

from datasets import load_dataset
from sklearn.model_selection import train_test_split
from transformers import pipeline, AutoModelForSequenceClassification, BertTokenizer, AutoTokenizer, \
                        TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments, AutoModelForCausalLM
from sklearn.metrics import classification_report

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Загрузим претренированную модель из зоопарка моделей HuggingFace.

In [3]:
model_name = 'cointegrated/rubert-base-cased-dp-paraphrase-detection'
model = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()
tokenizer = BertTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/712M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/406 [00:00<?, ?B/s]

Загрузим тестовый датасет, для которого будем строить предсказания.

In [4]:
corpus = load_dataset('merionum/ru_paraphraser', data_files='test.jsonl')
corpus

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/merionum___json/merionum--ru_paraphraser-57cfc8af008bb08a/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 1924
    })
})

In [5]:
def compare_texts(text1, text2):
    batch = tokenizer(text1, text2, return_tensors='pt').to(model.device)
    with torch.inference_mode():
        proba = torch.softmax(model(**batch).logits, -1).cpu().numpy()
    return np.argmax(proba[0]) # 0 - не парафраз, 1 - парафраз

In [6]:
classes = np.array(list(map(int, corpus['train']['class'])))

# Преобразуем классы в датасете - в исходном датасете три класса, а модель имеет бинарный выход

# 1: precise paraphrases
# 0: near paraphrases
# -1: non-paraphrases

classes = np.where(classes > -1, 1, 0)

Попробуем не обучать дополнительно модель, а сразу сделать предсказания.

In [7]:
%%time
preds = [compare_texts(row['text_1'], row['text_2']) for row in corpus['train']]    

CPU times: user 21.6 s, sys: 165 ms, total: 21.7 s
Wall time: 24.5 s


Выведем матрицу с метриками.

In [8]:
print(classification_report(classes, preds))

              precision    recall  f1-score   support

           0       0.86      0.75      0.80       772
           1       0.85      0.92      0.88      1152

    accuracy                           0.85      1924
   macro avg       0.85      0.83      0.84      1924
weighted avg       0.85      0.85      0.85      1924



Как видим модель дает довольно неплохо решает задачу определения парафраза из коробки, без дополнительного обучения.

2. Сначала загрузим датасет

In [9]:
dataset = load_dataset("sberquad")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/45328 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5036 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/23936 [00:00<?, ? examples/s]

Dataset sberquad downloaded and prepared to /root/.cache/huggingface/datasets/sberquad/sberquad/1.0.0/62115d937acf2634cfacbfee10c13a7ee39df3ce345bb45af7088676f9811e77. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
dataset['train']

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 45328
})

In [11]:
print(f"Вопрос: {dataset['train']['question'][5]}")
print(f"Ответ: {dataset['train']['answers'][5]['text'][0]}")

Вопрос: Куда поступают остатки непереваренной пищи?
Ответ: в толстый кишечник


Создадим файлы с текстами для обучения и валидации.

In [12]:
def build_text_files(texts, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for i, t in enumerate(texts):
        # будем использовать только первый ответ, в случае если их несколько
        data += 'Q: ' + t['question'] + ' A: '+ t['answers']['text'][0] + "   "
    f.write(data)

In [13]:
!rm -rf train_dataset.txt val_dataset.txt

train_path = 'train_dataset.txt'
val_path = 'val_dataset.txt'

build_text_files(dataset['train'], train_path)
build_text_files(dataset['validation'], val_path)

Создадим датасеты.

In [14]:
def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    val_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=val_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, val_dataset, data_collator

Импортируем модель и токенайзер среднего размера GPT3.

In [15]:
model_name = 'sberbank-ai/rugpt3medium_based_on_gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

train_dataset, val_dataset, data_collator = load_dataset(train_path, val_path, tokenizer)

Downloading:   0%|          | 0.00/674 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/1.73G [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


Задаем гиперпараметры и обучаем модель.

In [16]:
training_args = TrainingArguments(
    output_dir="./gpt3-qa", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )

In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

In [18]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 8638
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 6480


Step,Training Loss
500,2.740100
1000,2.629200
1500,2.589300
2000,2.573700
2500,2.298000
3000,2.169800
3500,2.168900
4000,2.168600
4500,2.049100
5000,1.873000


Saving model checkpoint to ./gpt3-qa/checkpoint-800
Configuration saved in ./gpt3-qa/checkpoint-800/config.json
Model weights saved in ./gpt3-qa/checkpoint-800/pytorch_model.bin
Saving model checkpoint to ./gpt3-qa/checkpoint-1600
Configuration saved in ./gpt3-qa/checkpoint-1600/config.json
Model weights saved in ./gpt3-qa/checkpoint-1600/pytorch_model.bin
Saving model checkpoint to ./gpt3-qa/checkpoint-2400
Configuration saved in ./gpt3-qa/checkpoint-2400/config.json
Model weights saved in ./gpt3-qa/checkpoint-2400/pytorch_model.bin
Saving model checkpoint to ./gpt3-qa/checkpoint-3200
Configuration saved in ./gpt3-qa/checkpoint-3200/config.json
Model weights saved in ./gpt3-qa/checkpoint-3200/pytorch_model.bin
Saving model checkpoint to ./gpt3-qa/checkpoint-4000
Configuration saved in ./gpt3-qa/checkpoint-4000/config.json
Model weights saved in ./gpt3-qa/checkpoint-4000/pytorch_model.bin
Saving model checkpoint to ./gpt3-qa/checkpoint-4800
Configuration saved in ./gpt3-qa/checkpoint-4

TrainOutput(global_step=6480, training_loss=2.2222558669102046, metrics={'train_runtime': 3561.3799, 'train_samples_per_second': 7.276, 'train_steps_per_second': 1.82, 'total_flos': 6016587449499648.0, 'train_loss': 2.2222558669102046, 'epoch': 3.0})

Сохраняем модель.

In [19]:
trainer.save_model()
tokenizer.save_pretrained('gpt3_qa')
model.save_pretrained('model_gpt3_qa')

Saving model checkpoint to ./gpt3-qa
Configuration saved in ./gpt3-qa/config.json
Model weights saved in ./gpt3-qa/pytorch_model.bin
tokenizer config file saved in gpt3_qa/tokenizer_config.json
Special tokens file saved in gpt3_qa/special_tokens_map.json
Configuration saved in model_gpt3_qa/config.json
Model weights saved in model_gpt3_qa/pytorch_model.bin


Загружаем токенайзер и модель из сохраненных файлов.

In [20]:
tokenizer_qa = AutoTokenizer.from_pretrained("gpt3_qa")
model_qa = AutoModelForCausalLM.from_pretrained("model_gpt3_qa")

loading file vocab.json
loading file merges.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file model_gpt3_qa/config.json
Model config GPT2Config {
  "_name_or_path": "model_gpt3_qa",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 1024,
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "n_positions": 2048,
  "n_special": 0,
  "output_past": true,
  "predict_special_tokens": true,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dro

Попробуем взять вопросы из тестовой выборки и получить на них ответы.

In [34]:
def generate_answer(prefix):
    tokens = tokenizer_qa(prefix, return_tensors='pt')
    size = tokens['input_ids'].shape[1]
    output = model_qa.generate(
        **tokens, 
#         eos_token="   ",
        do_sample=False,
        max_length=size+8, 
        repetition_penalty=5., 
        temperature=0.5,
        num_beams=10,
    )

    decoded = tokenizer_qa.decode(output[0])
    result = decoded[len(prefix):]
    print(prefix + result)

In [36]:
prefix = 'Q: ' + dataset['test']['question'][42]
generate_answer(prefix)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Q: Что необходимо для достижения оптимального результата? A: учитывать множество факторов   Q:


In [37]:
prefix = 'Q: ' + dataset['test']['question'][420]
generate_answer(prefix)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Q: После чего Жозе Сарамаго стал лауреатом Нобелевской премии по литературе 1998 года? A: после публикации романа Война и мир


In [38]:
prefix = 'Q: ' + dataset['test']['question'][100]
generate_answer(prefix)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Q: Когда Клузиус создал первую монографию, посвященную грибам? A: в 1788 году   Q


Как видим модель довольно неплохо справляется с генерацией ответов на поставленные вопросы.